## Step 5 : Upward sweep

In this notebook, we will exploit the advantage of tree structure to calculate the multipole for each cell.

In [1]:
import numpy
from treecode_helper import Particle, Cell, build_tree

In [2]:
n = 100            #number of particles
particles = [Particle(m = 1.0/n) for i in range(n)]
n_crit = 10        #max number of particles in a single cell.

In [3]:
root = Cell(n_crit)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

In [4]:
cells = build_tree(particles, root, n_crit)

In order to evaluate the potential at each target, we need to calculate the multipole of each cell on the tree (traverse the tree). Since we can obtain a cell's multipole by shifting all its children's multipoles (**M2M**), a natural way to traverse the tree is to start with calculating the multipoles of all leaf cells (**P2M**), and then sweep upward (from leaf cell to root cell) to implement **M2M**.

In [5]:
def get_multipole(particles, p, cells, n_crit):
    """Calculate multipole arrays for all leaf cells under cell p.
    If leaf number of cell p is equal or bigger than n_crit (non-leaf),
    traverse down recursively. Otherwise (leaf), calculate the multipole
    arrays for leaf cell p.
    
    Arguments:
        p: current cell's index
        cells: the list of cells
        particles: the array of all particles
        n_crit: maximum number of leaves in a single cell
    """
    
    # if the current cell p is not a leaf cell, then
    # recursively traverse down
    
    if cells[p].nleaf >= n_crit:
        
        for c in range(8): #checking all the octants
            
            if cells[p].nchild & (1 << c):
                
                get_multipole(particles, cells[p].child[c], cells, n_crit)
    else:
        # loop in leaf particles, do P2M
        for i in range(cells[p].nleaf):
            
            l = cells[p].leaf[i]
            dx, dy, dz = cells[p].x-particles[l].x, \
                         cells[p].y-particles[l].y, \
                         cells[p].z-particles[l].z
            
            cells[p].multipole += particles[l].m * \
                                  numpy.array((1, dx, dy, dz,\
                                               dx**2/2, dy**2/2, dz**2/2,\
                                               dx*dy/2, dy*dz/2, dz*dx/2)) 

In `get_multipole`, we first recursively traverse down from $p$-th cell to find the leaf cells among its descendants, and then do the P2M in the same way as we did in step 03. Knowing that the $1st$ element in `cells` is the root cell, we can calculate each leaf cell's multipole by setting $p=0$.

In [6]:
root = 0    # index of the root cell
get_multipole(particles, root, cells, n_crit)

In [7]:
def M2M(p, c, cells):
    """Calculate parent cell p's multipole array based on child cell c's 
    multipoles
    
    Arguments:
        p: parent cell index in cells list
        c: child cell index in cells list
        cells: the list of cells
    """
    dx, dy, dz = cells[p].x-cells[c].x, cells[p].y-cells[c].y, cells[p].z-cells[c].z
    
    Dxyz =  numpy.array((dx, dy, dz))
    Dyzx = numpy.roll(Dxyz,-1) #It permutes the array (dx,dy,dz) to (dy,dz,dx) 
    
    cells[p].multipole += cells[c].multipole
    
    cells[p].multipole[1:4] += cells[c].multipole[0] * Dxyz
    
    cells[p].multipole[4:7] += cells[c].multipole[1:4] * Dxyz\
                             + 0.5*cells[c].multipole[0] *  Dxyz**2
    
    cells[p].multipole[7:] += 0.5*numpy.roll(cells[c].multipole[1:4], -1) * Dxyz \
                            + 0.5*cells[c].multipole[1:4] * Dxyz \
                            + 0.5*cells[c].multipole[0] * Dxyz * Dyzx   

Recall that we contructed the tree in such fashion that the child of the $p$-th cell will be always located at the right side of $p$ in the list `cells`. Therefore, each leaf cell will be on the right side of its parents. Thus, one way to traverse the tree is looping from the tail to the head of the list `cells`.

In [8]:
def upward_sweep(cells):
    """Traverse from leaves to root, in order to calculate multipoles
    of all the cells.
    
    Arguments:
        cells: the list of cells
    
    """
    for c in range(len(cells)-1, 0, -1):
        p = cells[c].parent
        M2M(p, c, cells)

In [9]:
upward_sweep(cells)